<a href="https://colab.research.google.com/github/FRED984/healthcare_cost_prediction/blob/main/predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
label_dataset = dataset.pop('expenses')
label_dataset.head()

In [ ]:
from sklearn.model_selection import train_test_split

train_dataset, test_dataset, train_labels, test_labels = train_test_split(dataset, label_dataset, test_size=0.2, random_state=42)


In [ ]:
from sklearn.preprocessing import LabelEncoder

def transform_categorical_to_numerical(dataset):
  # Define your categorical columns
  CATEGORICAL_COLUMNS = ['sex', 'smoker', 'region']

  # Assuming df is your dataframe containing the data
  encoded_df = dataset.copy()  # Create a copy of the original dataframe

  # Initialize LabelEncoder
  label_encoders = {}

  # Encode each categorical column with LabelEncoder
  for col in CATEGORICAL_COLUMNS:
    label_encoders[col] = LabelEncoder()
    encoded_df[col] = label_encoders[col].fit_transform(dataset[col])
  return encoded_df

train_dataset = transform_categorical_to_numerical(train_dataset)
test_dataset = transform_categorical_to_numerical(test_dataset)
print(train_dataset)

In [ ]:
from sklearn.preprocessing import StandardScaler
# Standardize the features
scaler = StandardScaler()
train_dataset = scaler.fit_transform(train_dataset)
test_dataset = scaler.transform(test_dataset)

train_dataset = train_dataset.astype(np.float32)
test_dataset = test_dataset.astype(np.float32)

train_dataset = tf.convert_to_tensor(train_dataset)
train_labels = tf.convert_to_tensor(train_labels)
test_dataset = tf.convert_to_tensor(test_dataset)
test_labels = tf.convert_to_tensor(test_labels)


In [ ]:
#def make_input_fn (data_df,label_df, num_epochs=20, shuffle=True, batch_size=32):
  #def input_function():
    #ds = tf.data.Dataset.from_tensor_slices((dict(data_df),label_df))
    #if shuffle:
      #ds = ds.shuffle(1000)
    #ds =ds.batch(batch_size).repeat(num_epochs)
    #return ds
  #return input_function

In [ ]:
#train_input_fn = make_input_fn(train_dataset,train_label)
#model = tf.estimator.LinearClassifier(feature_columns=feature_columns)
#model.train(train_input_fn)

In [ ]:

# Build the neural network model
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=[train_dataset.shape[1]]),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='linear')  # Linear activation for regression
])

# Compile the model with appropriate loss function
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae', 'mse'])

# Train the model with validation data
model.fit(train_dataset, train_labels, epochs=50, batch_size=10, validation_data=(test_dataset, test_labels))



In [ ]:
loss= model.evaluate(test_dataset, test_labels)
print(loss)

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
